In [60]:
import sys
import os
import HTSeq

In [61]:
import numpy

In [62]:
sys.version

'3.6.7 | packaged by conda-forge | (default, Feb 20 2019, 02:51:38) \n[GCC 7.3.0]'

In [63]:
os.chdir("/rhome/jluis/bigdata/RNA-seq")

In [64]:
os.getcwd()

'/bigdata/castaneralab/jluis/RNA-seq'

In [65]:
#chr2 = HTSeq.FileOrSequence('Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz')

In [66]:
chr2 = HTSeq.FastaReader('Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz')
chr2.fos

'Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz'

In [67]:
for seq in chr2:
    print("Sequence '%s' has length %d." % ( seq.name, len(seq) ))

Sequence '8' has length 145138636.


In [68]:
seq[222100:222200].seq

b'CCAGGAGTTCAACACCAGCCAGAAGGACATAGCAAAACCCCGTCTCTACTAAACATACAAAAATTAGTCGTGCATGGTGACACACACCTGTAGTCACAGC'

In [69]:
length_chr8 = len(seq) 
length_chr8

145138636

In [70]:
type(seq)

HTSeq._HTSeq.Sequence

In [71]:
type(seq.seq)

bytes

In [72]:
print(seq.seq[3], seq.seq[3:4])

78 b'N'


In [73]:
for nuc in seq.seq[:3]:
    print(nuc)

78
78
78


In [74]:
for nuc in seq.seq[:3]:
    print(chr(nuc))

N
N
N


In [75]:
print(seq.seq[3:4])

b'N'


### Counting frequencies of each nucleotide 

`HTSeq.base_to_column` is part of `HTSeq`

In [76]:
HTSeq.base_to_column

{'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}

### So, to count bases, we can make and array of 5 columns representing  ‘A’, ‘C’, ‘G’, ‘T’, and ‘N’ where each column index comes from   `HTSeq.base_to_column`


In [77]:
import numpy as np
counts = np.zeros((length_chr8,5), np.int)
seq.add_bases_to_count_array(counts)


In [78]:
counts[222110:222120, : ]

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [79]:
counts[:, HTSeq.base_to_column["C"]]/ counts.sum(1)

array([0., 0., 0., ..., 0., 0., 0.])

In [80]:
counts.nbytes

5805545440

In [81]:
sys.getsizeof(counts)/ (1e9) #1000 Megabytes, 1GB

5.805545552

In [82]:
#%store counts

In [83]:
#%store 

### Taking into account the ambigous bases (Ns)

In [84]:
counts[:, HTSeq.base_to_column["A"]].sum() / length_chr8 

0.29856646854528796

In [85]:
for k in HTSeq.base_to_column:
    print(counts[:, HTSeq.base_to_column[k]].sum() / length_chr8)

0.29856646854528796
0.20001685147433795
0.2005240492958746
0.298339898963912
0.0025527317205874802


In [86]:
# fitht column has the N counts
Ns = counts[: , 4].sum()
Ns

370500

### Without Ns

In [87]:
for k in HTSeq.base_to_column:
    print(counts[:, HTSeq.base_to_column[k]].sum() / (length_chr8 - Ns))

0.29933057920977857
0.2005287475691474
0.20103724344423415
0.29910342977683985
0.002559264837118577


**Comparing with BBMap stats module ** 

`stats.sh  Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz > bbmap.stats`

In [88]:
!head -3 bbmap.stats

A	C	G	T	N	IUPAC	Other	GC	GC_stdev
0.2993	0.2005	0.2010	0.2991	0.0026	0.0000	0.0000	0.4016	0.0000



### Computing GC content from the `count` array.

`HTSeq.base_to_column = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}`


In [89]:
Gs = counts[: , 2] == 1 
Cs = counts[: , 1] == 1

GC_content = (Gs.sum() + Cs.sum()) / (length_chr8 - Ns)

In [90]:
GC_content

0.4015659910133816

### Number of  CpGs 

Every row is a nucleotide position in the sequence, so adjacent rows are adjacent bases.

We have to find all the occurrences where a row has an `1` in its 2nd column followed by a row with an `1` in the 3rd column. 

In [91]:
Cs_idxs = np.where(counts[: , 1] == 1)  
print(Cs_idxs , Cs_idxs[0].shape)

(array([    60001,     60010,     60012, ..., 145078615, 145078616,
       145078628]),) (29030173,)


**29 million of Cs in the human chromosome 8**

### `np.where()` returns a tuple of arrays, in this case is an one element tuple with an array containing the 1st index (row index) where the condition is `True`. It has one element because in the comparison the array is sliced taking only the second column (index = 1)

**the array has to be unpacked first, then add 1 to take the nucleotide ahead of every `C`. that nucleotide has to be a `G`, i.e. the 3rd column equal to 1**

In [92]:
CpG_mask = counts[(Cs_idxs[0]+1), 2] == 1
CpG_mask

array([False, False, False, ..., False, False, False])

In [93]:
CpGs_number = CpG_mask.sum()
CpGs_number

1338200

*** ~ 1.34 million of CpGs in the human chromosome 8***

### To get the positions of these CpGs, we can use `np.where`. This will return CpGs positions relative to the `Cs_idxs` array

In [94]:
CpGs = np.where(counts[(Cs_idxs[0]+1), 2] == 1)
CpGs[0].shape

(1338200,)

In [95]:
CpGs

(array([      66,       70,       84, ..., 29030109, 29030118, 29030159]),)

In [96]:
Cs_idxs[0][CpGs[0][:3]]

array([60406, 60421, 60497])

In [97]:
counts[Cs_idxs[0][CpGs[0][:3]]]

array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

**Those are all Cs but followed by Gs:**

In [98]:
counts[Cs_idxs[0][CpGs[0][:3]] + 1]

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]])

In [99]:
print(seq.seq[60404:60408], seq.seq[60419:60423] )

b'AACG' b'CACG'


## Using Regex 

In [100]:
import re

In [101]:
pattern = re.compile(b'CG')

In [102]:
len(re.findall(pattern, seq.seq))

1338200

In [103]:
for match in re.finditer(pattern, seq.seq[:62000]):
    print(match.span(), match.group())

(60406, 60408) b'CG'
(60421, 60423) b'CG'
(60497, 60499) b'CG'
(60552, 60554) b'CG'
(60780, 60782) b'CG'
(60860, 60862) b'CG'
(60886, 60888) b'CG'
(60925, 60927) b'CG'
(61046, 61048) b'CG'
(61063, 61065) b'CG'
(61100, 61102) b'CG'
(61613, 61615) b'CG'
(61631, 61633) b'CG'
(61642, 61644) b'CG'
(61712, 61714) b'CG'
(61736, 61738) b'CG'
(61787, 61789) b'CG'
(61883, 61885) b'CG'


**This confirms the results from the previous approach using `numpy`**

In [104]:
#!free -m

## Using biopython

In [108]:
from Bio import SeqIO
from Bio.Alphabet import IUPAC
import gzip

In [109]:
with gzip.open("Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz", "rt") as handle:
    seq_iterator = SeqIO.parse(handle, "fasta")
    #print(sum(len(r) for r in seq_iterator))
    record = list(seq_iterator)[0]

File is closed after this, so I have to get all the info inside the `with` statment. In this case we get only a single element list (it's one chromosome)

In [110]:
type(record)

Bio.SeqRecord.SeqRecord

In [111]:
print(record.format, record.description, record.name, record.id, sep='\n\n')

<bound method SeqRecord.format of SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet()), id='8', name='8', description='8 dna:chromosome chromosome:GRCh38:8:1:145138636:1 REF', dbxrefs=[])>

8 dna:chromosome chromosome:GRCh38:8:1:145138636:1 REF

8

8


In [112]:
record.seq.alphabet

SingleLetterAlphabet()

In [113]:
with gzip.open("Homo_sapiens.GRCh38.dna.chromosome.8.fa.gz", "rt") as handle:
    seq_iterator = SeqIO.parse(handle, format="fasta", alphabet=IUPAC.IUPACAmbiguousDNA())
    #print(sum(len(r) for r in seq_iterator))
    record_alphabet = list(seq_iterator)[0]

In [114]:
record_alphabet.seq.alphabet

IUPACAmbiguousDNA()

In [115]:
from Bio.SeqUtils import GC

In [116]:
GC(record.seq)

40.05409007702126

In [117]:
GC(record_alphabet.seq)

40.05409007702126

```python
GC?
```
>_Calculates G+C content, returns the percentage (float between 0 and 100).
Copes mixed case sequences, and with the ambiguous nucleotide S (G or C)
when counting the G and C content_.  **The percentage is calculated against
the full length:**


```python
from Bio.SeqUtils import GC
GC("ACTGN")
40.0```

2 of 5 = 0.4 ----> **It counts Ns**

In [118]:
os.chdir("/bigdata/castaneralab/jluis/Tutorials/Comparing_genomes")

In [119]:
ls -ltrh

total 86M
drwxr-xr-x 5 jluis castaneralab 4.0K Nov 12  2018 Panseq/
-rw-r--r-- 1 jluis castaneralab  15K Nov 13  2018 README_assembly_summary.txt
drwxr-xr-x 3 jluis castaneralab 4.0K Nov 13  2018 Mauve/
-rw-r--r-- 1 jluis castaneralab  42M Nov 13  2018 assembly_summary_refseq.txt
-rw-r--r-- 1 jluis castaneralab  275 Nov 13  2018 genomes_summary.awk
-rw-r--r-- 1 jluis castaneralab 6.4K Nov 13  2018 E_coli_chromosome_level.list
-rw-r--r-- 1 jluis castaneralab 1.5M Nov 13  2018 GCF_002741235.1_ASM274123v1_genomic.fna.gz
-rw-r--r-- 1 jluis castaneralab 1.5M Nov 13  2018 GCF_002764155.1_ASM276415v1_genomic.fna.gz
-rw-r--r-- 1 jluis castaneralab 359K Nov 13  2018 GCF_002741235.1_ASM274123v1_genomic.gff.gz
-rw-r--r-- 1 jluis castaneralab 354K Nov 13  2018 GCF_002764155.1_ASM276415v1_genomic.gff.gz
drwxr-xr-x 2 jluis castaneralab 4.0K Nov 13  2018 Mummer/
-rw-r--r-- 1 jluis castaneralab 1.4M Nov 15  2018 GCF_0027412358.1_ASM274123v1_genomic_chromosome.fasta.gz
-rw-r--r-- 1 jluis castaneralab 1

In [120]:
import re

In [121]:
ecoli_1 = HTSeq.FastaReader('GCF_002741235.1_ASM274123v1_genomic.fna.gz')

for seq in ecoli_1:
    print("Sequence '%s | %s' has length %d." % ( seq.name, seq.descr, len(seq) ))

Sequence 'NZ_CP024299.1 | Escherichia coli strain 90-9276 chromosome' has length 5004671.
Sequence 'NZ_CP024297.1 | Escherichia coli strain 90-9276 plasmid unnamed1' has length 167764.
Sequence 'NZ_CP024298.1 | Escherichia coli strain 90-9276 plasmid unnamed2' has length 110577.


In [122]:
seq.seq[:100]

b'TGCGTTAACGATGTAATGAATCAGAACACGCACCTGCTCGTCTGAAATTTCCACCATGACCTGCGGGAGTTTGTCCAGCAGCTCCATCATGTCACGATGG'

### In `htseq`, the seq class is bytes, so it has to be decoded into a string  

**Creating fasta files with "the" chromosome (these genomes also have plasmids)**

In [123]:
ecoli_1 = HTSeq.FastaReader('GCF_002741235.1_ASM274123v1_genomic.fna.gz')

pattern = re.compile(r'chromosome', re.I)

for seq in ecoli_1:
    if re.search(pattern, seq.descr):
        print("Sequence '%s | %s' has length %d." % ( seq.name, seq.descr, len(seq) ))
        with open('GCF_0027412358.1_ASM274123v1_genomic_chromosome80.fasta', mode='w', encoding='utf8') as fh:
            #print('>{0} {1}'.format(seq.name, seq.descr), seq.seq.decode(encoding='utf8'), sep='\n',file=fh)
            seq.write_to_fasta_file(fh, 80)

Sequence 'NZ_CP024299.1 | Escherichia coli strain 90-9276 chromosome' has length 5004671.


In [124]:
ecoli_1 = HTSeq.FastaReader('GCF_002764155.1_ASM276415v1_genomic.fna.gz')

pattern = re.compile(r'chromosome', re.I)

for seq in ecoli_1:
    if re.search(pattern, seq.descr):
        print("Sequence '%s | %s' has length %d." % ( seq.name, seq.descr, len(seq) ))
        with open('GCF_002764155.1_ASM276415v1_genomic_chromosome80.fasta', mode='w', encoding='utf8') as fh:
            #print('>{0} {1}'.format(seq.name, seq.descr), seq.seq.decode(encoding='utf8'), sep='\n',file=fh)
            seq.write_to_fasta_file(fh, 80)

Sequence 'NZ_CP024661.1 | Escherichia coli strain 90-9269 chromosome' has length 4760041.


In [125]:
! awk '{print length($0)}' GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta

57
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

In [126]:
!head -n 5 GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta

>NZ_CP024661.1 Escherichia coli strain 90-9269 chromosome
GCTGGCGGTGCCGATGATGATGGGCGCAACGGCGCTCGATCTCTACAAAAGCTGGGGCTTCCTGACAAGCGGCGATATCC
CGATGTTTGCCGTTGGGTTTATCACCGCTTTTGTGGTGGCGCTGATAGCGATTAAAACCTTCCTGCAATTGATTAAGCGC
ATTTCGTTTATCCCGTTCGCCATTTATCGCTTTATTGTGGCGGCTGCGGTGTATGTCGTGTTCTTTTAATTGAGGAATAG
TGTCGGATGCTTGGCGTTGTCAACGATTGCGTAGGCCGAATAAGGCGTTCACGCCGCATCCGGCAATCAACGCCTGATGC


In [127]:
fasta = '/bigdata/castaneralab/jluis/Tutorials/Comparing_genomes/Mummer/H_pylori26695_Eslice.fasta'
with open(fasta, "rt") as handle:
    seq_iterator = SeqIO.parse(handle, format="fasta", alphabet=IUPAC.IUPACAmbiguousDNA())
    #print(sum(len(r) for r in seq_iterator))
    record_alphabet = list(seq_iterator)[0]

In [128]:
record_alphabet.features,record_alphabet.format, record_alphabet.seq

([],
 <bound method SeqRecord.format of SeqRecord(seq=Seq('TTAATTTTAGAAATACAGGTTTCTAAAACGCTTTTATGCGGCTCGCCCTCATAG...AAG', IUPACAmbiguousDNA()), id='H_pylori26695_Eslice', name='H_pylori26695_Eslice', description='H_pylori26695_Eslice', dbxrefs=[])>,
 Seq('TTAATTTTAGAAATACAGGTTTCTAAAACGCTTTTATGCGGCTCGCCCTCATAG...AAG', IUPACAmbiguousDNA()))

In [129]:
pwd

'/bigdata/castaneralab/jluis/Tutorials/Comparing_genomes'

In [130]:
seq.seq[:10]

b'CATTCCCCGG'

In [131]:
ls

assembly_summary_refseq.txt
E_coli_chromosome_level.list
GCF_000181755.1_ASM18175v1_genomic.fna.gz
GCF_000181775.1_ASM18177v1_genomic.fna.gz
GCF_000181775.1_ASM18177v1_genomic.gbff.gz
GCF_002741235.1_ASM274123v1_genomic.fna.gz
GCF_002741235.1_ASM274123v1_genomic.gbff.gz
GCF_002741235.1_ASM274123v1_genomic.gff.gz
GCF_0027412358.1_ASM274123v1_genomic_chromosome80.fasta
GCF_0027412358.1_ASM274123v1_genomic_chromosome.fasta
GCF_0027412358.1_ASM274123v1_genomic_chromosome.fasta.gz
GCF_002764155.1_ASM276415v1_genomic_chromosome80.fasta
GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta
GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta.gz
GCF_002764155.1_ASM276415v1_genomic.fna.gz
GCF_002764155.1_ASM276415v1_genomic.gbff.gz
GCF_002764155.1_ASM276415v1_genomic.gff.gz
genomes_summary.awk
Mauve/
Mummer/
nucmer.coords
nucmer.delta
Panseq/
README_assembly_summary.txt


In [132]:
fasta1 ="GCF_002764155.1_ASM276415v1_genomic_chromosome80.fasta"
ecoli_1 = HTSeq.FastaReader(fasta1)

pattern = re.compile(r'chromosome', re.I)

for seq in ecoli_1:
    print("Sequence '%s | %s' has length %d." % ( seq.name, seq.descr, len(seq) ))


Sequence 'NZ_CP024661.1 | Escherichia coli strain 90-9269 chromosome' has length 4760041.


In [133]:
ls

assembly_summary_refseq.txt
E_coli_chromosome_level.list
GCF_000181755.1_ASM18175v1_genomic.fna.gz
GCF_000181775.1_ASM18177v1_genomic.fna.gz
GCF_000181775.1_ASM18177v1_genomic.gbff.gz
GCF_002741235.1_ASM274123v1_genomic.fna.gz
GCF_002741235.1_ASM274123v1_genomic.gbff.gz
GCF_002741235.1_ASM274123v1_genomic.gff.gz
GCF_0027412358.1_ASM274123v1_genomic_chromosome80.fasta
GCF_0027412358.1_ASM274123v1_genomic_chromosome.fasta
GCF_0027412358.1_ASM274123v1_genomic_chromosome.fasta.gz
GCF_002764155.1_ASM276415v1_genomic_chromosome80.fasta
GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta
GCF_002764155.1_ASM276415v1_genomic_chromosome.fasta.gz
GCF_002764155.1_ASM276415v1_genomic.fna.gz
GCF_002764155.1_ASM276415v1_genomic.gbff.gz
GCF_002764155.1_ASM276415v1_genomic.gff.gz
genomes_summary.awk
Mauve/
Mummer/
nucmer.coords
nucmer.delta
Panseq/
README_assembly_summary.txt


In [134]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

In [135]:
for seq_record in SeqIO.parse(fasta1, format="fasta", 
                              alphabet=IUPAC.unambiguous_dna):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record.description))

    print(len(seq_record))

NZ_CP024661.1
Seq('GCTGGCGGTGCCGATGATGATGGGCGCAACGGCGCTCGATCTCTACAAAAGCTG...NNN', IUPACUnambiguousDNA())
56
4760041
